In [2]:
import torch
import numpy as np

In [26]:
def img2col(input, h_out, w_out, h_k, w_k, stride):
    """
        Stacking ONLY Single Channel
    :input: (batch, channel, height, weight)
    :return: (batch, channel, h_k*w_k, h_out*w_out)
    """
    b, c, h, w = input.shape
    out = np.zeros((b, c, h_k*w_k, h_out*w_out))
    convhIdx = 0
    convwIdx = 0
    for i in range(b):
        for j in range(c):
            # For each channel, scan from left-top
            convwIdx = 0
            convhIdx = 0
            for k in range(h_out*w_out):
                if convwIdx + w_k > w:
                    convwIdx = 0
                    convhIdx += stride
                out[i, j, :, k] = input[i, j, convhIdx:convhIdx+h_k, convwIdx:convwIdx+w_k].flatten()
                convwIdx += stride
    return out


In [43]:
def forward(input, weights, bias, pad=2, stride=1, in_channel=1, out_channel=1):
    """
    # Arguments
        input: numpy array with shape (batch, in_channel, in_height, in_width)
        weights: numpy array with shape (out_channel, in_channel, kernel_h, kernel_w)
        bias: numpy array with shape (out_channel)

    # Returns
        output: numpy array with shape (batch, out_channel, out_height, out_width)
    """
    kernel_h = weights.shape[2] #self.conv_params['kernel_h']  # height of kernel
    kernel_w = weights.shape[3] #self.conv_params['kernel_w']  # width of kernel
    # pad = 0 #self.conv_params['pad']
    # stride = 1 #self.conv_params['stride']
    # in_channel = 1# self.conv_params['in_channel']
    # out_channel = 1# self.conv_params['out_channel']

    batch, in_channel, in_height, in_width = input.shape
    #####################################################################################
    # computing output shape
    out_h = int((in_height + pad - kernel_h)/stride) + 1
    out_w = int((in_width + pad - kernel_w)/stride) + 1

    # padding
    pad_input = np.pad(input, ((0,0),(0,0),(int(pad/2), pad-int(pad/2)),(int(pad/2), pad-int(pad/2))), 'constant', constant_values=0)
    in_height += pad
    in_width += pad

    # Img2Col
    col_input = img2col(pad_input, out_h, out_w, kernel_h, kernel_w, stride)
    # merge channel
    col_input = col_input.reshape(col_input.shape[0], -1, col_input.shape[3])

    # reshape kernel
    weights_flatten = weights.reshape(weights.shape[0], -1)
    
    # compute convolution
    output = weights_flatten @ col_input + bias.reshape(-1, 1)

    # reshape convolution result
    output = output.reshape(output.shape[0], output.shape[1], out_h, out_w)
    #####################################################################################
    return output


In [38]:
conv2d = torch.nn.Conv2d(1,1,3,1,1)
conv2d.weight.data.fill_(1)
conv2d.bias.data.fill_(1)
print('Weight Of Network :\n',conv2d.weight)
print('Bias Of Network :\n',conv2d.bias)

Weight Of Network :
 Parameter containing:
tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]], requires_grad=True)
Bias Of Network :
 Parameter containing:
tensor([1.], requires_grad=True)


In [44]:
test_input_int = [[[[2,0,0,3,7],[3,4,6,1,6],[7,8,8,5,8],[8,6,4,4,6],[8,6,9,8,4]]]]
test_input = torch.Tensor(test_input_int)
out = conv2d(test_input)

print(f"Model Input")
print(test_input)
print("Model Output")
print(out)
img2col_out = forward(np.array(test_input_int),conv2d.weight.detach().numpy(),conv2d.bias.detach().numpy())
print("Img2col Output")
print(img2col_out)

Model Input
tensor([[[[2., 0., 0., 3., 7.],
          [3., 4., 6., 1., 6.],
          [7., 8., 8., 5., 8.],
          [8., 6., 4., 4., 6.],
          [8., 6., 9., 8., 4.]]]])
Model Output
tensor([[[[10., 16., 15., 24., 18.],
          [25., 39., 36., 45., 31.],
          [37., 55., 47., 49., 31.],
          [44., 65., 59., 57., 36.],
          [29., 42., 38., 36., 23.]]]], grad_fn=<ConvolutionBackward0>)
Img2col Output
[[[[10. 16. 15. 24. 18.]
   [25. 39. 36. 45. 31.]
   [37. 55. 47. 49. 31.]
   [44. 65. 59. 57. 36.]
   [29. 42. 38. 36. 23.]]]]


In [35]:
img2col_out = forward(test_input_int,conv2d.weight.detach().numpy(),conv2d.bias.detach().numpy())
print("Img2col Output")
print(img2col_out)

Img2col Output
[[[[4. 2.]
   [4. 2.]]]]


In [ ]:
weight_to_write = new_weight_int
bias_to_write = new_bias_int
input_to_write = test_input_int
golden_to_write = [[int(x) for x in row] for row in out.tolist()]

In [ ]:
weight_file = "weight.csv"
bias_file = "bias.csv"
input_file = "input.csv"
golden_file = "golden.csv"

import csv
with open(weight_file, 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(weight_to_write)
with open(bias_file, 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(bias_to_write)
with open(input_file, 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(input_to_write)
with open(golden_file, 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(golden_to_write)